In [1]:
import numpy as np
import pandas as pd
import os
import easygui
import openpyxl
import re

In [2]:
def readexcel(fname):
    df = pd.read_excel(fname,dtype = {'订单号':'str','Offer_ID':'str', 'SKU_ID':'str'})
    return df

def alibabaXiaoshouMingxi(fname):
    df = pd.read_excel(fname,dtype = {'订单号':'str','Offer_ID':'str', 'SKU_ID':'str'})
    col1s = ['订单号','卖家公司名称', '卖家会员名','订单状态', '订单创建时间', '订单付款时间', '收货人姓名', '收货地址', '邮编',
       '联系手机',  '货运公司', '运单号']
    col2s = ['货品总价', '运费（元）', '商家改价（元）',
           '实付款（元）','单价']
    for col in col1s:
        df[col] =  df[col].ffill()
    for col in col2s :
        df[col] = df[col].fillna(0)
    return df

def gethuohao(df0,df1):
    df1 = df1[~df1['订单号'].isin(df0['订单号'].to_list())]
    df = pd.concat([df0,df1])
    return df
    
def getZhuangtai(df):
    # df = df[(df['订单状态'] == '已收货') | (df['订单状态'] == '交易成功')]
    df = df[df['订单状态'] == '交易成功']
    return df

fname0 = r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\阿里巴巴订单中没有品名(1).xlsx"
fname = r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\阿里巴巴202240726-20241031.xlsx"
df0 = readexcel(fname0)
df1 = alibabaXiaoshouMingxi(fname)
df = gethuohao(df0,df1)
df = getZhuangtai(df)
df_dingdan = df[['订单号','单品货号','数量','货品总价','运费（元）','商家改价（元）','实付款（元）','单价']]
df_dingdan['jiner'] = df_dingdan['数量']*df_dingdan['单价']
df_dingdan



C:\Users\redda\AppData\Local\Temp\ipykernel_32872\1125658386.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan['jiner'] = df_dingdan['数量']*df_dingdan['单价']


,订单号,单品货号,数量,货品总价,运费（元）,商家改价（元）,实付款（元）,单价,jiner
0,4029296580454477008,EXLA5100Pa,5,10.55,2.8,0.0,13.35,2.11,10.55
1,2278067991350911471,EXLB5100Pa,240,691.20,91.2,0.0,782.40,2.88,691.20
4,2275086218060301182,EXLA6100P,2,4.86,2.8,0.0,7.66,0.57,1.14
5,2275086218060301182,EXBA7100P,3,0.00,0.0,0.0,0.00,1.24,3.72
6,4022437645929203749,XJ404S,900,6109.60,425.0,-1593.6,4941.00,0.61,549.00
...,...,...,...,...,...,...,...,...,...
1635,4024897020184462145,ETCG1630N,1,2.63,2.8,0.0,5.43,0.75,0.75
1636,4024897020184462145,EJCG1630N（分区）,2,0.00,0.0,0.0,0.00,0.94,1.88
1637,2275754989406013772,RJRJ-1680-G01,10,35.90,5.8,0.0,41.70,3.59,35.90
1658,2269186502496746753,EJL3280,10,15.60,4.3,0.0,19.90,1.56,15.60


In [3]:
pivot = df_dingdan.pivot_table(index = '单品货号',values = ['数量','货品总价','运费（元）','商家改价（元）','实付款（元）','jiner'] ,aggfunc = 'sum')
pivot

,jiner,商家改价（元）,实付款（元）,数量,货品总价,运费（元）
单品货号,,,,,,
16K备课本-30P,27.0,0.0,53.7,30,40.6,13.1
16K备课本-50P,13.6,0.0,0.0,10,0.0,0.0
2002,18.9,0.0,0.0,10,0.0,0.0
2003,18.9,0.0,0.0,10,0.0,0.0
2004,18.9,0.0,0.0,10,0.0,0.0
...,...,...,...,...,...,...
XJ402S,488.0,0.0,0.0,800,0.0,0.0
XJ403S,549.0,0.0,0.0,900,0.0,0.0
XJ404S,549.0,-1593.6,4941.0,900,6109.6,425.0


In [4]:
#支付宝收入
fname_shouru = r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\2088340986968423-20241105-141006887-收入.csv"
df_shouru = pd.read_csv(fname_shouru,header = 2,encoding = 'gb18030',dtype = {'商户订单号':'str','业务基础订单号':'str', '支付宝交易号':'str','支付宝流水号':'str',})
df_shouru = df_shouru[~df_shouru['支付宝交易号'].isnull()]
df_shouru = df_shouru[['入账时间', '支付宝交易号', '支付宝流水号', '商户订单号', '账务类型', '收入（+元）', '支出（-元）',
             '备注', '业务基础订单号', '业务订单号']]
df_shouru['收入（+元）'] = df_shouru['收入（+元）'].fillna(0)
df_shouru['收入（+元）'] = df_shouru['收入（+元）'].replace(' ',0)
df_shouru['支出（-元）'] = df_shouru['支出（-元）'].fillna(0)
df_shouru['支出（-元）'] = df_shouru['支出（-元）'].replace(' ',0)
df_shouru1 = df_shouru[df_shouru['账务类型'].isin(['其它', '转账', '在线支付', '分账', '退款', '退款（交易退款）']) ]
df_shouru1 = df_shouru1.astype({'入账时间':'datetime64[ns]'})
# df_shouru1 = df_shouru[~(df_shouru['账务类型'] == '充值（大额充值）')]
df_shouru1['收入（+元）'] = df_shouru1['收入（+元）'].astype('float64') 
df_shouru1['支出（-元）'] = df_shouru1['支出（-元）'].astype('float64') 
df_shouru1['jinshouru'] = df_shouru1['收入（+元）'] - df_shouru1['支出（-元）']
df_shouru1

,入账时间,支付宝交易号,支付宝流水号,商户订单号,账务类型,收入（+元）,支出（-元）,备注,业务基础订单号,业务订单号,jinshouru
0,2024-10-31 20:35:00,2024102822001863911432047748,1370588720884563911,T50060NP2346736370492300393,在线支付,7.20,0.00,,2346736370492300393,M{2346736370492300393},7.20
1,2024-10-31 15:41:00,2024102122001816521438174872,1370664944675716521,T50060NP2334027253383968950,在线支付,123.85,0.00,,2334027253383968950,M{2334027253383968950},123.85
2,2024-10-31 15:39:00,2024102122001886621455162759,1370569179927686621,T50060NP4088655146461741737,在线支付,308.50,0.00,,4088655146461741737,M{4088655146461741737},308.50
3,2024-10-31 15:39:00,2024102122001868991443403167,1370591121145668991,T50060NP4088298961495140808,在线支付,12.80,0.00,,4088298961495140808,M{4088298961495140808},12.80
4,2024-10-31 15:38:00,2024102122001819311427812929,1370627415990419311,T50060NP4085497225241684439,在线支付,6.30,0.00,,4085497225241684439,M{4085497225241684439},6.30
...,...,...,...,...,...,...,...,...,...,...,...
1109,2024-10-05 19:28:00,2024100510002001420568939674,1370604592252668420,202410050125100000004200728381793,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_2305884720464935473,,,-0.01
1110,2024-10-05 11:17:00,2024100510002001420568945256,1370604318296168420,202410050125100000004200726514909,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_4056386329560742647,,,-0.01
1111,2024-09-30 10:28:00,2024093010002001420567119117,1370602199102968420,202409300125100000004200706904713,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_4050833510009863504,,,-0.01
1112,2024-09-28 13:18:00,2024092810002001420566281993,1370601575932268420,202409280125100000004200699577192,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_2295939180500400752,,,-0.01


In [ ]:
df_shouru1.to_excel('shouru1-13.xlsx',index = False)

In [5]:
regex1 = re.compile(r'^1688.*_(\d+)')  
regex2 = re.compile(r'^1688.*-订单号(\d+)')
regex3 = re.compile(r'^1688.*[:\]](\d+)-.*')
regex4 = re.compile(r'^聚宝盆.*订单号:(\d+).*')
regex5 = re.compile(r'^代扣款.*订单号(\d+).*')

# string = r'T50060NP2268563451685578186'
# mat = re.search(regex1,string)
# mat
def getDingdanhao(str1,str2):
    if str1 ==  None:
        return str1
    else :
        try:
            mat = re.search(regex1,str2)
            return mat.group(1)
        except:
             try:
                mat = re.search(regex2,str2)
                return mat.group(1)
             except:
                 try:
                    mat = re.search(regex3,str2)
                    return mat.group(1)
                 except:
                     try:
                        mat = re.search(regex4,str2)
                        return mat.group(1)
                     except:
                         try:
                            mat = re.search(regex5,str2)
                            return mat.group(1)
                         except:
                             return str1
                         
               
  
        
df_shouru2 =  df_shouru1.assign(dingdanhao = df_shouru1.apply(lambda x:getDingdanhao(x['业务基础订单号'],x['备注']),axis = 1))

df_shouru2

# string = r'聚宝盆严选_超链商家收款_订单号:2333778961387277854(2333778961387277854)'
# mat = re.search(regex3,string)
# mat

# mat.group(1)    
    


,入账时间,支付宝交易号,支付宝流水号,商户订单号,账务类型,收入（+元）,支出（-元）,备注,业务基础订单号,业务订单号,jinshouru,dingdanhao
0,2024-10-31 20:35:00,2024102822001863911432047748,1370588720884563911,T50060NP2346736370492300393,在线支付,7.20,0.00,,2346736370492300393,M{2346736370492300393},7.20,2346736370492300393
1,2024-10-31 15:41:00,2024102122001816521438174872,1370664944675716521,T50060NP2334027253383968950,在线支付,123.85,0.00,,2334027253383968950,M{2334027253383968950},123.85,2334027253383968950
2,2024-10-31 15:39:00,2024102122001886621455162759,1370569179927686621,T50060NP4088655146461741737,在线支付,308.50,0.00,,4088655146461741737,M{4088655146461741737},308.50,4088655146461741737
3,2024-10-31 15:39:00,2024102122001868991443403167,1370591121145668991,T50060NP4088298961495140808,在线支付,12.80,0.00,,4088298961495140808,M{4088298961495140808},12.80,4088298961495140808
4,2024-10-31 15:38:00,2024102122001819311427812929,1370627415990419311,T50060NP4085497225241684439,在线支付,6.30,0.00,,4085497225241684439,M{4085497225241684439},6.30,4085497225241684439
...,...,...,...,...,...,...,...,...,...,...,...,...
1109,2024-10-05 19:28:00,2024100510002001420568939674,1370604592252668420,202410050125100000004200728381793,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_2305884720464935473,,,-0.01,2305884720464935473
1110,2024-10-05 11:17:00,2024100510002001420568945256,1370604318296168420,202410050125100000004200726514909,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_4056386329560742647,,,-0.01,4056386329560742647
1111,2024-09-30 10:28:00,2024093010002001420567119117,1370602199102968420,202409300125100000004200706904713,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_4050833510009863504,,,-0.01,4050833510009863504
1112,2024-09-28 13:18:00,2024092810002001420566281993,1370601575932268420,202409280125100000004200699577192,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_2295939180500400752,,,-0.01,2295939180500400752


In [ ]:
# regex4 = re.compile(r'^聚宝盆.*订单号:(\d+).*')
# string = r'聚宝盆严选_超链商家收款_订单号:2297277699764277854(2297277699764277854)'
# mat = re.search(regex4,string)
# mat

# mat.group(1)    
    

In [6]:
df_shouru2['dingdanhao'] = df_shouru2['dingdanhao'].fillna('8888888888888888888')
df_shouru2['dingdanhao'] = df_shouru2['dingdanhao'].astype('str')
df_shouru2['dingdanhao'] = df_shouru2['dingdanhao'].replace(' ','8888888888888888888')
df_shouru2

,入账时间,支付宝交易号,支付宝流水号,商户订单号,账务类型,收入（+元）,支出（-元）,备注,业务基础订单号,业务订单号,jinshouru,dingdanhao
0,2024-10-31 20:35:00,2024102822001863911432047748,1370588720884563911,T50060NP2346736370492300393,在线支付,7.20,0.00,,2346736370492300393,M{2346736370492300393},7.20,2346736370492300393
1,2024-10-31 15:41:00,2024102122001816521438174872,1370664944675716521,T50060NP2334027253383968950,在线支付,123.85,0.00,,2334027253383968950,M{2334027253383968950},123.85,2334027253383968950
2,2024-10-31 15:39:00,2024102122001886621455162759,1370569179927686621,T50060NP4088655146461741737,在线支付,308.50,0.00,,4088655146461741737,M{4088655146461741737},308.50,4088655146461741737
3,2024-10-31 15:39:00,2024102122001868991443403167,1370591121145668991,T50060NP4088298961495140808,在线支付,12.80,0.00,,4088298961495140808,M{4088298961495140808},12.80,4088298961495140808
4,2024-10-31 15:38:00,2024102122001819311427812929,1370627415990419311,T50060NP4085497225241684439,在线支付,6.30,0.00,,4085497225241684439,M{4085497225241684439},6.30,4085497225241684439
...,...,...,...,...,...,...,...,...,...,...,...,...
1109,2024-10-05 19:28:00,2024100510002001420568939674,1370604592252668420,202410050125100000004200728381793,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_2305884720464935473,,,-0.01,2305884720464935473
1110,2024-10-05 11:17:00,2024100510002001420568945256,1370604318296168420,202410050125100000004200726514909,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_4056386329560742647,,,-0.01,4056386329560742647
1111,2024-09-30 10:28:00,2024093010002001420567119117,1370602199102968420,202409300125100000004200706904713,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_4050833510009863504,,,-0.01,4050833510009863504
1112,2024-09-28 13:18:00,2024092810002001420566281993,1370601575932268420,202409280125100000004200699577192,其它,0.00,0.01,1688买家保障_增值买家保障软件服务费_2295939180500400752,,,-0.01,2295939180500400752


In [ ]:
df_shouru2.to_excel('shouru2-8.xlsx',index = False)

In [7]:
pivot_shouru = pd.pivot_table(df_shouru2,index = 'dingdanhao',values = ['收入（+元）','支出（-元）','jinshouru'] ,aggfunc = 'sum')
pivot_shouru = pivot_shouru.reset_index()
result = pd.merge(pivot_shouru,df_dingdan,left_on = 'dingdanhao',right_on = '订单号',how = 'left')
result['订单号'] = result['订单号'].fillna('8888888888888888888')
result['单品货号'] = result['单品货号'].fillna('A8888')
result = result.assign(jiner = np.where(result['单品货号'] == 'A8888',result['jinshouru'],result.jiner))
result

   


,dingdanhao,jinshouru,支出（-元）,收入（+元）,订单号,单品货号,数量,货品总价,运费（元）,商家改价（元）,实付款（元）,单价,jiner
0,2212249947371705472,-0.50,0.50,0.00,8888888888888888888,A8888,NaN,NaN,NaN,NaN,NaN,NaN,-0.50
1,2268563451685578186,70.76,0.36,71.12,2268563451685578186,EXBA5100NB,36.0,60.12,11.0,0.0,71.12,1.67,60.12
2,2269186502496746753,19.81,0.09,19.90,2269186502496746753,EJL3280,10.0,15.60,4.3,0.0,19.90,1.56,15.60
3,2273049049602822267,115.41,0.59,116.00,2273049049602822267,RFDF1660N,100.0,88.00,17.0,11.0,116.00,0.88,88.00
4,2275086218060301182,7.63,0.03,7.66,2275086218060301182,EXLA6100P,2.0,4.86,2.8,0.0,7.66,0.57,1.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,4098104930457304908,-0.44,0.44,0.00,8888888888888888888,A8888,NaN,NaN,NaN,NaN,NaN,NaN,-0.44
1057,4099734686905080632,25.38,0.12,25.50,4099734686905080632,EJCG1630N,10.0,19.00,6.5,0.0,25.50,0.94,9.40
1058,4099734686905080632,25.38,0.12,25.50,4099734686905080632,ETCG1640N,10.0,0.00,0.0,0.0,0.00,0.96,9.60
1059,4100516282085368020,24.58,0.12,24.70,4100516282085368020,FYP-1660,10.0,20.20,4.5,0.0,24.70,2.02,20.20


In [ ]:
result.to_excel('shouru_dingdan分配前-1.xlsx',index = False)

In [9]:
gp = result.groupby('订单号')
#将订单总金额按各产品的明细销售额进行分配
data = []
for k,v in gp:
    # print(k,v)
    mean = v['jinshouru'].mean()
    total = v['jiner'].sum()
    # print('mean',mean)
    # print('total',total)
    v['result_jiner'] = v['jinshouru']*(v.jiner)/total
    data.append(v)
result1 = pd.concat(data)
# result1 = result1.reset_index()

result1

,dingdanhao,jinshouru,支出（-元）,收入（+元）,订单号,单品货号,数量,货品总价,运费（元）,商家改价（元）,实付款（元）,单价,jiner,result_jiner
1,2268563451685578186,70.76,0.36,71.12,2268563451685578186,EXBA5100NB,36.0,60.12,11.0,0.0,71.12,1.67,60.12,70.760000
2,2269186502496746753,19.81,0.09,19.90,2269186502496746753,EJL3280,10.0,15.60,4.3,0.0,19.90,1.56,15.60,19.810000
3,2273049049602822267,115.41,0.59,116.00,2273049049602822267,RFDF1660N,100.0,88.00,17.0,11.0,116.00,0.88,88.00,115.410000
4,2275086218060301182,7.63,0.03,7.66,2275086218060301182,EXLA6100P,2.0,4.86,2.8,0.0,7.66,0.57,1.14,1.789753
5,2275086218060301182,7.63,0.03,7.66,2275086218060301182,EXBA7100P,3.0,0.00,0.0,0.0,0.00,1.24,3.72,5.840247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,4086758415098573717,4.50,0.00,4.50,8888888888888888888,A8888,NaN,NaN,NaN,NaN,NaN,NaN,4.50,0.061206
1042,4087610787961742525,-0.08,0.08,0.00,8888888888888888888,A8888,NaN,NaN,NaN,NaN,NaN,NaN,-0.08,0.000019
1052,4094415613346651910,-0.05,0.05,0.00,8888888888888888888,A8888,NaN,NaN,NaN,NaN,NaN,NaN,-0.05,0.000008
1056,4098104930457304908,-0.44,0.44,0.00,8888888888888888888,A8888,NaN,NaN,NaN,NaN,NaN,NaN,-0.44,0.000585


In [10]:
result2 = result1[['dingdanhao', 'jinshouru',  '单品货号', '数量','单价', 'jiner', 'result_jiner']]
result2.columns = ['订单号', '净收入',  '单品货号', '数量','单价',  '金额', '最终金额']
result2 = result2.assign(最终金额 = np.where(result2['单品货号'] == 'A8888',result2['净收入'],result2['最终金额']))
# pivot = pivot.reset_index()
total =result2.sum().to_frame().T
result2 = pd.concat([result2,total])
result2.iloc[-1,0] = '小计'
result2.iloc[-1,1] = ''
result2.iloc[-1,2] = ''

result2

,订单号,净收入,单品货号,数量,单价,金额,最终金额
1,2268563451685578186,70.76,EXBA5100NB,36.0,1.67,60.12,70.76
2,2269186502496746753,19.81,EJL3280,10.0,1.56,15.6,19.81
3,2273049049602822267,115.41,RFDF1660N,100.0,0.88,88.0,115.41
4,2275086218060301182,7.63,EXLA6100P,2.0,0.57,1.14,1.789753
5,2275086218060301182,7.63,EXBA7100P,3.0,1.24,3.72,5.840247
...,...,...,...,...,...,...,...
1042,4087610787961742525,-0.08,A8888,NaN,NaN,-0.08,-0.08
1052,4094415613346651910,-0.05,A8888,NaN,NaN,-0.05,-0.05
1056,4098104930457304908,-0.44,A8888,NaN,NaN,-0.44,-0.44
1060,8888888888888888888,275.4,A8888,NaN,NaN,275.4,275.4


In [11]:
result2.to_excel('阿里巴巴销售明细表按品名汇总0726-1031-08-11.xlsx',index = False)